In [29]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm


df = pd.read_excel('台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl')
display(df.head(3))
display(df[df['品名']=='MA'])

產品集合 = set(df['品名'].values)

'''
train_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)
train_df = train_df.dropna(axis=0)
display(train_df.head(3))

val_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)
val_df = val_df.dropna(axis=0)
display(val_df.head(3))
'''

,RIGID,分機,公司代號,公司事業部門,品名,新增
0,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID FLAKE PEARLS MICROPEARLS,NaN
1,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID,NaN
2,黃淑玲,7190,11,台塑塑膠部,COMMODITY FORMOSACN,NaN


,RIGID,分機,公司代號,公司事業部門,品名,新增
531,NaN,NaN,2P,南亞塑四部化學品部,MA,NaN


"\ntrain_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)\ntrain_df = train_df.dropna(axis=0)\ndisplay(train_df.head(3))\n\nval_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)\nval_df = val_df.dropna(axis=0)\ndisplay(val_df.head(3))\n"

In [30]:
val_df = pd.read_csv('preprocess_for_SQUAD_wordninja.csv',index_col=0)[['45A','Y_label']]
val_df.columns = ['string_X_train','Y_label']
val_df

,string_X_train,Y_label
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX
8,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010
9,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010
...,...,...
6671,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL
6672,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER
6673,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN
6674,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN


In [31]:
train_df = val_df

# 如果品名是單詞的話 前後加個空白

In [32]:
for i in df.index:
    if ' ' not in df.loc[i,'品名']:
        name = df.loc[i,'品名']
        df.loc[i,'品名'] = f' {name} '
        assert df.loc[i,'品名'][0] == ' '
        assert df.loc[i,'品名'][-1] == ' '

In [33]:
display(df[df['品名']=='MA'])
display(df[df['品名']==' MA '])

,RIGID,分機,公司代號,公司事業部門,品名,新增


,RIGID,分機,公司代號,公司事業部門,品名,新增
531,NaN,NaN,2P,南亞塑四部化學品部,MA,NaN


In [34]:
產品集合 = set(df['品名'].values)

# find_fail_sample and drop fail_sample

In [35]:
def find_fail_sample(df):
    fails = []
    for i in df.index:
        context = df.loc[i,'string_X_train']
        answer = df.loc[i,'Y_label']
        if str(answer) not in str(context):
            fails.append(i)
    return fails
train_fails = find_fail_sample(train_df)
val_fails = find_fail_sample(val_df)
print(train_fails,val_fails)
display(val_df.loc[val_fails])
print(val_df.shape)
val_df = val_df.drop(val_fails,axis=0)
print(val_df.shape)

[233, 662, 663, 702, 1066, 1069, 1095, 1096, 1097, 1234, 1246, 1247, 1248, 1249, 1250, 1325, 1326, 1327, 1344, 1370, 1371, 1399, 1400, 1407, 1409, 1458, 1459, 1460, 1461, 1592, 1596, 1598, 1599, 1602, 1605, 1612, 1619, 1620, 1640, 1642, 1643, 1756, 1758, 1848, 1911, 1914, 1915, 1939, 1940, 1941, 2016, 2017, 2023, 2138, 2184, 2186, 2187, 2206, 2208, 2218, 2305, 2348, 2468, 2469, 2492, 2493, 2518, 2589, 2590, 2646, 2725, 2800, 2809, 2866, 2867, 2868, 2869, 2871, 2872, 2914, 2915, 2916, 2919, 2921, 2924, 2985, 2986, 3019, 3102, 3103, 3232, 3233, 3418, 3420, 3421, 3612, 3613, 3868, 3870, 3887, 3912, 3917, 3940, 3987, 4085, 4121, 4165, 4167, 4173, 4174, 4175, 4397, 4405, 4430, 4480, 4522, 4594, 4595, 4596, 4632, 4639, 4650, 4983, 5039, 5057, 5191, 5290, 5292, 5293, 5298, 5304, 5305, 5306, 5307, 5312, 5313, 5318, 5443, 5509, 5537, 5538, 5539, 5560, 5561, 5590, 5594, 5595, 5730, 5731, 5732, 5733, 5821, 5877, 5895, 5944, 5945, 6005, 6006, 6007, 6016, 6017, 6018, 6019, 6020, 6032, 6062, 6151, 6

,string_X_train,Y_label
233,PHTHALIC ANHYDRIDE 504 MT AT USD 1010 MT x 000...,NaN
662,108 MT PHTHALIC ANHYDRIDE x 000 Dx 000 DC IF M...,NaN
663,108 MT PHTHALIC ANHYDRIDE x 000 Dx 000 DC IF M...,NaN
702,PHTHALIC ANHYDRIDE 504 MT AT USD 980 MT x 000 ...,NaN
1066,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN
...,...,...
6649,PHTHALIC ANHYDRIDE PAx 000 D Q TY 54 M TSx 000...,NaN
6656,108 MT 6 FC L OF PHTHALIC ANHYDRIDE AT 98000 P...,NaN
6666,108 MT 6 FC L OF PHTHALIC ANHYDRIDE AT USD 960...,NaN
6669,CI F JEBEL ALI UAE x 000 D PHTHALIC ANHYDRIDE ...,NaN


(5849, 2)
(5667, 2)


In [36]:
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            if p in df.loc[i,'string_X_train']:
                products.append(p)
        labels[i] = products
    predict = pd.DataFrame(index=labels.keys(),columns=['predict'])
    predict['predict'] = labels.values()
    return predict
predict = Collection_method(val_df,產品集合)
result = val_df.join(predict)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/5667 [00:00<?, ?it/s]

In [37]:
result

,string_X_train,Y_label,predict
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,"[ RESIN , PVC RESIN, PVC RESIN B-57]"
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,"[PHTHALIC ANHYDRIDE PA, PA , PHTHALIC ANHYDRI..."
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,"[ TAISOX , LLDPE TAISOX]"
8,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,"[HDPE TAISOX, HDPE TAISOX 8010, TAISOX , HDPE ]"
9,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,"[HDPE TAISOX, HDPE TAISOX 8010, TAISOX , HDPE ]"
...,...,...,...
6671,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,"[ ETHYLENE , MONO ETHYLENE GLYCOL]"
6672,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,"[VISCOSE RAYON STAPLE FIBER, RAYON , RAYON ST..."
6673,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,"[ RESIN , PC RESIN, INA ]"
6674,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,"[ RESIN , PC RESIN, INA ]"


In [38]:
not_find = []
for j,i in enumerate(result.iloc[:,2].values):
    if len(i) == 0:
        not_find.append(j)
len(not_find)

502

In [39]:
result.iloc[not_find]

,string_X_train,Y_label,predict
65,PVC SUSPENSION S65 - 112 MT AT USD 133000 PER ...,EA,[]
187,EVA 7470 Mx 000 D QUANTITY 5000 M TSx 000 DUN ...,EVA,[]
188,EVA 7470 Mx 000 D QUANTITY 5000 M TSx 000 DUN ...,EVA,[]
189,EVA 7470 Mx 000 D QUANTITY 5000 M TSx 000 DUN ...,EVA,[]
262,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,[]
...,...,...,...
6622,ABS AG 15 AA - H x 000 D Q U A N T I T Y 25200...,ABS,[]
6647,COMMODITY MONOETHYLENE GLYCOL x 000 D QUANTITY...,ETHYLENE,[]
6657,EPICHLOROHYDRIN x 000 D Q TY 18400 x 000 DAS P...,EPICHLOROHYDRIN,[]
6664,ACETONITRILE 128 MT x 000 DAT USD 3650 PER MT ...,ACETONITRILE,[]


In [40]:
def get_acc(df):
    correct = []
    correct_label = []
    for i in df.index:
        if df.loc[i,'Y_label'] in df.loc[i,'predict']:
            correct.append('yes')
        else:
            correct.append('no')
    return correct

In [41]:
correct = get_acc(result)
result['correct'] = correct
result

,string_X_train,Y_label,predict,correct
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,"[ RESIN , PVC RESIN, PVC RESIN B-57]",no
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,"[PHTHALIC ANHYDRIDE PA, PA , PHTHALIC ANHYDRI...",no
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,"[ TAISOX , LLDPE TAISOX]",yes
8,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,"[HDPE TAISOX, HDPE TAISOX 8010, TAISOX , HDPE ]",yes
9,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,"[HDPE TAISOX, HDPE TAISOX 8010, TAISOX , HDPE ]",yes
...,...,...,...,...
6671,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,"[ ETHYLENE , MONO ETHYLENE GLYCOL]",yes
6672,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,"[VISCOSE RAYON STAPLE FIBER, RAYON , RAYON ST...",yes
6673,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,"[ RESIN , PC RESIN, INA ]",yes
6674,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,"[ RESIN , PC RESIN, INA ]",yes


In [42]:
result['correct'].value_counts()

yes    3580
no     2087
Name: correct, dtype: int64

In [43]:
result['correct'].value_counts()['yes']/len(val_df)

0.6317275454385036

# 放水acc

In [48]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def get_acc2(df):
    correct = []
    correct_label = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        try:
            if max(jacs) >= 0.75:
                correct.append('yes')
            else:
                correct.append('no')
        except:
            correct.append('no')
    return correct

In [50]:
a = pd.Series(get_acc2(result))
a.value_counts()['yes']/len(a)

0.8041291688724193

# jac

In [55]:
import numpy as np
def get_jac(df):
    all_jacs = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        try:
            all_jacs.append(max(jacs))
        except:
            pass
    return np.sum(all_jacs)/len(all_jacs)

In [56]:
get_jac(result)

0.9299119531646153